## Expression prediction via Factor analysis

This notebook means to demonstrate how failure data can be used to predict success of protein expression experiments.
For this, we chose a factor analysis model (basically a Bayesian variant of PCA), since inserting prior knowledge can accelarate inference.
Also, its simplicity means it is well-suited for a proof-of-concept type of work, which really is the purpose of this document. 

Note that factor analysis assumes latent variates are normally distributed.
If data can not be explained with normally distributed variables, or you know the process in question is not Gaussian, factor analysis is expected to fail you.

In [14]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import scipy as sp
import sqlite3
import seaborn as sns
import xarray as xr

from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from numpy.random import default_rng
from xarray_einstats import linalg
from xarray_einstats.stats import XrContinuousRV

print(f"Running on PyMC v{pm.__version__}")

Running on PyMC v5.26.1


In [15]:
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

np.set_printoptions(precision=3, suppress=True)
RANDOM_SEED = 8942
rng = default_rng(RANDOM_SEED)

In [16]:
# Import failure experiment data

with sqlite3.connect("../lab_data.db") as con:
    cur = con.cursor()
    res = cur.execute("SELECT name FROM sqlite_master")
    print(res.fetchone())
    for i, table in enumerate(res.fetchone()):
        print(i, table)
    query_result = pd.read_sql_query("SELECT * FROM samples", con)
    df = pd.DataFrame(query_result)
    print(df.head())

('samples',)
0 sqlite_sequence
   id sample_id researcher expressed                        date  \
0   1                             No  2026-01-14T13:49:58.004075   
1   2                             No  2026-01-14T13:49:58.009325   
2   3                             No  2026-01-14T13:49:58.015248   
3   4                             No  2026-01-14T13:49:58.019705   
4   5                             No  2026-01-14T13:49:58.024319   

            created_at  
0  2026-01-14 13:49:58  
1  2026-01-14 13:49:58  
2  2026-01-14 13:49:58  
3  2026-01-14 13:49:58  
4  2026-01-14 13:49:58  


In [ ]:
# Number of latent variables
# TODO Settle latent variable number, make it dep. on PCA?
lat_vars = 4
obs_col = 10 # TODO Update dep. 
rows = len(data)

coords = {
    "latent_columns": np.arange(k),
    "rows": np.arange(n),
    "observed_columns": np.arange(d),
}

with pm.Model(coords=coords) as PPCA:
    W = pm.Normal("W", dims=("observed_columns", "latent_columns"))
    F = pm.Normal("F", dims=("latent_columns", "rows"))
    sigma = pm.HalfNormal("sigma", 1.0)
    X = pm.Normal(
        "X",
        mu=W @ F,
        sigma=sigma,
        observed=Y,
        dims=("observed_columns", "rows"),
    )

    trace = pm.sample(tune=2000, random_seed=rng)  # target_accept=0.9

NameError: name 'data' is not defined